<a href="https://colab.research.google.com/github/Nilanjan-Chatterjee/Aahar/blob/main/ensembled_faster_rcnn_fnac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Starting**

In [ ]:
#connecting the drive to colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import cv2

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
!git clone https://github.com/harshatejas/pytorch_custom_object_detection.git
%cd /content/pytorch_custom_object_detection
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import os
import torch
import glob
from torch.utils.data import Dataset
import time
import copy
import numpy as np
import math
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from torch.utils.data.dataset import Dataset  # For custom data-sets
import torchvision.transforms as transforms
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
import argparse
import logging
import sys
import cv2
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import random
from google.colab.patches import cv2_imshow

Cloning into 'pytorch_custom_object_detection'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 447 (delta 17), reused 13 (delta 13), pack-reused 428
Receiving objects: 100% (447/447), 36.73 MiB | 39.02 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/pytorch_custom_object_detection


# **Model**

In [ ]:
from torch import nn
import torch.nn.functional as F

from torchvision.ops import MultiScaleRoIAlign

from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.anchor_utils import AnchorGenerator

from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN
from torchvision.models.detection.roi_heads import RoIHeads
from torchvision.models.detection.rpn import RPNHead, RegionProposalNetwork
from torchvision.models.detection.transform import GeneralizedRCNNTransform
#from torchvision.models.detection.backbone_utils import  _resnet_fpn_extractor, _validate_trainable_layers, _mobilenet_extractor
#from .backbone_utils import resnet_fpn_backbone, _validate_trainable_layers, mobilenet_backbone
import numpy as np
import pandas as pd
import cv2

import torch
import torchvision
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
!git clone https://github.com/harshatejas/pytorch_custom_object_detection.git
%cd /content/pytorch_custom_object_detection
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import os
import torch
import glob
from torch.utils.data import Dataset
import time
import copy
import numpy as np
import math
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from torch.utils.data.dataset import Dataset  # For custom data-sets
import torchvision.transforms as transforms
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
import argparse
import logging
import sys
import cv2
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import random
from google.colab.patches import cv2_imshow

Cloning into 'pytorch_custom_object_detection'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 447 (delta 17), reused 13 (delta 13), pack-reused 428
Receiving objects: 100% (447/447), 36.73 MiB | 40.40 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/pytorch_custom_object_detection


In [ ]:
class FasterRCNN(GeneralizedRCNN):
    """
    Implements Faster R-CNN.

    The input to the model is expected to be a list of tensors, each of shape [C, H, W], one for each
    image, and should be in 0-1 range. Different images can have different sizes.

    The behavior of the model changes depending if it is in training or evaluation mode.

    During training, the model expects both the input tensors, as well as a targets (list of dictionary),
    containing:
        - boxes (``FloatTensor[N, 4]``): the ground-truth boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (Int64Tensor[N]): the class label for each ground-truth box

    The model returns a Dict[Tensor] during training, containing the classification and regression
    losses for both the RPN and the R-CNN.

    During inference, the model requires only the input tensors, and returns the post-processed
    predictions as a List[Dict[Tensor]], one for each input image. The fields of the Dict are as
    follows:
        - boxes (``FloatTensor[N, 4]``): the predicted boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (Int64Tensor[N]): the predicted labels for each image
        - scores (Tensor[N]): the scores or each prediction

    Args:
        backbone (nn.Module): the network used to compute the features for the model.
            It should contain a out_channels attribute, which indicates the number of output
            channels that each feature map has (and it should be the same for all feature maps).
            The backbone should return a single Tensor or and OrderedDict[Tensor].
        num_classes (int): number of output classes of the model (including the background).
            If box_predictor is specified, num_classes should be None.
        min_size (int): minimum size of the image to be rescaled before feeding it to the backbone
        max_size (int): maximum size of the image to be rescaled before feeding it to the backbone
        image_mean (Tuple[float, float, float]): mean values used for input normalization.
            They are generally the mean values of the dataset on which the backbone has been trained
            on
        image_std (Tuple[float, float, float]): std values used for input normalization.
            They are generally the std values of the dataset on which the backbone has been trained on
        rpn_anchor_generator (AnchorGenerator): module that generates the anchors for a set of feature
            maps.
        rpn_head (nn.Module): module that computes the objectness and regression deltas from the RPN
        rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
        rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
        rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
        rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
        rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals
        rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
            considered as positive during training of the RPN.
        rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
            considered as negative during training of the RPN.
        rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
            for computing the loss
        rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
            of the RPN
        rpn_score_thresh (float): during inference, only return proposals with a classification score
            greater than rpn_score_thresh
        box_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
            the locations indicated by the bounding boxes
        box_head (nn.Module): module that takes the cropped feature maps as input
        box_predictor (nn.Module): module that takes the output of box_head and returns the
            classification logits and box regression deltas.
        box_score_thresh (float): during inference, only return proposals with a classification score
            greater than box_score_thresh
        box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
        box_detections_per_img (int): maximum number of detections per image, for all classes.
        box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
            considered as positive during training of the classification head
        box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
            considered as negative during training of the classification head
        box_batch_size_per_image (int): number of proposals that are sampled during training of the
            classification head
        box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
            of the classification head
        bbox_reg_weights (Tuple[float, float, float, float]): weights for the encoding/decoding of the
            bounding boxes

    Example::

        >>> import torch
        >>> import torchvision
        >>> from torchvision.models.detection import FasterRCNN
        >>> from torchvision.models.detection.rpn import AnchorGenerator
        >>> # load a pre-trained model for classification and return
        >>> # only the features
        >>> backbone = torchvision.models.mobilenet_v2(pretrained=True).features
        >>> # FasterRCNN needs to know the number of
        >>> # output channels in a backbone. For mobilenet_v2, it's 1280
        >>> # so we need to add it here
        >>> backbone.out_channels = 1280
        >>>
        >>> # let's make the RPN generate 5 x 3 anchors per spatial
        >>> # location, with 5 different sizes and 3 different aspect
        >>> # ratios. We have a Tuple[Tuple[int]] because each feature
        >>> # map could potentially have different sizes and
        >>> # aspect ratios
        >>> anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
        >>>                                    aspect_ratios=((0.5, 1.0, 2.0),))
        >>>
        >>> # let's define what are the feature maps that we will
        >>> # use to perform the region of interest cropping, as well as
        >>> # the size of the crop after rescaling.
        >>> # if your backbone returns a Tensor, featmap_names is expected to
        >>> # be ['0']. More generally, the backbone should return an
        >>> # OrderedDict[Tensor], and in featmap_names you can choose which
        >>> # feature maps to use.
        >>> roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
        >>>                                                 output_size=7,
        >>>                                                 sampling_ratio=2)
        >>>
        >>> # put the pieces together inside a FasterRCNN model
        >>> model = FasterRCNN(backbone,
        >>>                    num_classes=2,
        >>>                    rpn_anchor_generator=anchor_generator,
        >>>                    box_roi_pool=roi_pooler)
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)
    """

    def __init__(self, backbone, num_classes=None,
                 # transform parameters
                 min_size=800, max_size=1333,
                 image_mean=None, image_std=None,
                 # RPN parameters
                 rpn_anchor_generator=None, rpn_head=None,
                 rpn_pre_nms_top_n_train=2000, rpn_pre_nms_top_n_test=1000,
                 rpn_post_nms_top_n_train=2000, rpn_post_nms_top_n_test=1000,
                 rpn_nms_thresh=0.7,
                 rpn_fg_iou_thresh=0.7, rpn_bg_iou_thresh=0.3,
                 rpn_batch_size_per_image=256, rpn_positive_fraction=0.5,
                 rpn_score_thresh=0.0,
                 # Box parameters
                 box_roi_pool=None, box_head=None, box_predictor=None,
                 box_score_thresh=0.05, box_nms_thresh=0.5, box_detections_per_img=500,
                 box_fg_iou_thresh=0.5, box_bg_iou_thresh=0.5,
                 box_batch_size_per_image=512, box_positive_fraction=0.25,
                 bbox_reg_weights=None):
        print("hello")
        if not hasattr(backbone, "out_channels"):
            raise ValueError(
                "backbone should contain an attribute out_channels "
                "specifying the number of output channels (assumed to be the "
                "same for all the levels)")

        assert isinstance(rpn_anchor_generator, (AnchorGenerator, type(None)))
        assert isinstance(box_roi_pool, (MultiScaleRoIAlign, type(None)))

        if num_classes is not None:
            if box_predictor is not None:
                raise ValueError("num_classes should be None when box_predictor is specified")
        else:
            if box_predictor is None:
                raise ValueError("num_classes should not be None when box_predictor "
                                 "is not specified")

        out_channels = backbone.out_channels

        if rpn_anchor_generator is None:
            anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
            aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
            rpn_anchor_generator = AnchorGenerator(
                anchor_sizes, aspect_ratios
            )
        if rpn_head is None:
            rpn_head = RPNHead(
                out_channels, rpn_anchor_generator.num_anchors_per_location()[0]
            )

        rpn_pre_nms_top_n = dict(training=rpn_pre_nms_top_n_train, testing=rpn_pre_nms_top_n_test)
        rpn_post_nms_top_n = dict(training=rpn_post_nms_top_n_train, testing=rpn_post_nms_top_n_test)

        rpn = RegionProposalNetwork(
            rpn_anchor_generator, rpn_head,
            rpn_fg_iou_thresh, rpn_bg_iou_thresh,
            rpn_batch_size_per_image, rpn_positive_fraction,
            rpn_pre_nms_top_n, rpn_post_nms_top_n, rpn_nms_thresh,
            score_thresh=rpn_score_thresh)

        if box_roi_pool is None:
            box_roi_pool = MultiScaleRoIAlign(
                featmap_names=['0', '1', '2', '3'],
                output_size=7,
                sampling_ratio=2)

        if box_head is None:
            resolution = box_roi_pool.output_size[0]
            representation_size = 1024
            box_head = TwoMLPHead(
                out_channels * resolution ** 2,
                representation_size)

        if box_predictor is None:
            representation_size = 1024
            box_predictor = FastRCNNPredictor(
                representation_size,
                num_classes)

        roi_heads = RoIHeads(
            # Box
            box_roi_pool, box_head, box_predictor,
            box_fg_iou_thresh, box_bg_iou_thresh,
            box_batch_size_per_image, box_positive_fraction,
            bbox_reg_weights,
            box_score_thresh, box_nms_thresh, box_detections_per_img)

        if image_mean is None:
            image_mean = [0.485, 0.456, 0.406]
        if image_std is None:
            image_std = [0.229, 0.224, 0.225]
        transform = GeneralizedRCNNTransform(min_size, max_size, image_mean, image_std)

        super(FasterRCNN, self).__init__(backbone, rpn, roi_heads, transform)


class TwoMLPHead(nn.Module):
    """
    Standard heads for FPN-based models

    Args:
        in_channels (int): number of input channels
        representation_size (int): size of the intermediate representation
    """

    def __init__(self, in_channels, representation_size):
        super(TwoMLPHead, self).__init__()

        self.fc6 = nn.Linear(in_channels, representation_size)
        self.fc7 = nn.Linear(representation_size, representation_size)

    def forward(self, x):
        x = x.flatten(start_dim=1)

        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))

        return x


class FastRCNNPredictor(nn.Module):
    """
    Standard classification + bounding box regression layers
    for Fast R-CNN.

    Args:
        in_channels (int): number of input channels
        num_classes (int): number of output classes (including background)
    """

    def __init__(self, in_channels, num_classes):
        super(FastRCNNPredictor, self).__init__()
        self.cls_score = nn.Linear(in_channels, num_classes)
        self.bbox_pred = nn.Linear(in_channels, num_classes * 4)

    def forward(self, x):
        if x.dim() == 4:
            assert list(x.shape[2:]) == [1, 1]
        x = x.flatten(start_dim=1)
        scores = self.cls_score(x)
        bbox_deltas = self.bbox_pred(x)

        return scores, bbox_deltas


model_urls = {
    'fasterrcnn_resnet50_fpn_coco':
        'https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth',
    'fasterrcnn_mobilenet_v3_large_320_fpn_coco':
        'https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth',
    'fasterrcnn_mobilenet_v3_large_fpn_coco':
        'https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth'
}

def fasterrcnn_resnet50_fpn(pretrained=False, progress=True,
                            num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None, **kwargs):
    """
    Constructs a Faster R-CNN model with a ResNet-50-FPN backbone.

    Reference: `"Faster R-CNN: Towards Real-Time Object Detection with
    Region Proposal Networks" <https://arxiv.org/abs/1506.01497>`_.

    The input to the model is expected to be a list of tensors, each of shape ``[C, H, W]``, one for each
    image, and should be in ``0-1`` range. Different images can have different sizes.

    The behavior of the model changes depending if it is in training or evaluation mode.

    During training, the model expects both the input tensors, as well as a targets (list of dictionary),
    containing:

        - boxes (``FloatTensor[N, 4]``): the ground-truth boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (``Int64Tensor[N]``): the class label for each ground-truth box

    The model returns a ``Dict[Tensor]`` during training, containing the classification and regression
    losses for both the RPN and the R-CNN.

    During inference, the model requires only the input tensors, and returns the post-processed
    predictions as a ``List[Dict[Tensor]]``, one for each input image. The fields of the ``Dict`` are as
    follows, where ``N`` is the number of detections:

        - boxes (``FloatTensor[N, 4]``): the predicted boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (``Int64Tensor[N]``): the predicted labels for each detection
        - scores (``Tensor[N]``): the scores of each detection

    For more details on the output, you may refer to :ref:`instance_seg_output`.

    Faster R-CNN is exportable to ONNX for a fixed batch size with inputs images of fixed size.

    Example::

        >>> model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        >>> # For training
        >>> images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
        >>> labels = torch.randint(1, 91, (4, 11))
        >>> images = list(image for image in images)
        >>> targets = []
        >>> for i in range(len(images)):
        >>>     d = {}
        >>>     d['boxes'] = boxes[i]
        >>>     d['labels'] = labels[i]
        >>>     targets.append(d)
        >>> output = model(images, targets)
        >>> # For inference
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)
        >>>
        >>> # optionally, if you want to export the model to ONNX:
        >>> torch.onnx.export(model, x, "faster_rcnn.onnx", opset_version = 11)

    Args:
        pretrained (bool): If True, returns a model pre-trained on COCO train2017
        progress (bool): If True, displays a progress bar of the download to stderr
        num_classes (int): number of output classes of the model (including the background)
        pretrained_backbone (bool): If True, returns a model with backbone pre-trained on Imagenet
        trainable_backbone_layers (int): number of trainable (not frozen) resnet layers starting from final block.
            Valid values are between 0 and 5, with 5 meaning all backbone layers are trainable.
    """
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 5, 3)

    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False
    backbone = resnet_fpn_backbone('resnet50', pretrained_backbone, trainable_layers=trainable_backbone_layers)
    model = FasterRCNN(backbone, num_classes, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['fasterrcnn_resnet50_fpn_coco'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        overwrite_eps(model, 0.0)
    return model



def _fasterrcnn_mobilenet_v3_large_fpn(weights_name, pretrained=False, progress=True, num_classes=91,
                                       pretrained_backbone=True, trainable_backbone_layers=None, **kwargs):
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 6, 3)

    if pretrained:
        pretrained_backbone = False
    backbone = mobilenet_backbone("mobilenet_v3_large", pretrained_backbone, True,
                                  trainable_layers=trainable_backbone_layers)

    anchor_sizes = ((32, 64, 128, 256, 512, ), ) * 3
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)

    model = FasterRCNN(backbone, num_classes, rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios),
                       **kwargs)
    if pretrained:
        if model_urls.get(weights_name, None) is None:
            raise ValueError("No checkpoint is available for model {}".format(weights_name))
        state_dict = load_state_dict_from_url(model_urls[weights_name], progress=progress)
        model.load_state_dict(state_dict)
    return model

def fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=False, progress=True, num_classes=91, pretrained_backbone=True,
                                          trainable_backbone_layers=None, **kwargs):
    """
    Constructs a low resolution Faster R-CNN model with a MobileNetV3-Large FPN backbone tunned for mobile use-cases.
    It works similarly to Faster R-CNN with ResNet-50 FPN backbone. See
    :func:`~torchvision.models.detection.fasterrcnn_resnet50_fpn` for more
    details.

    Example::

        >>> model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)

    Args:
        pretrained (bool): If True, returns a model pre-trained on COCO train2017
        progress (bool): If True, displays a progress bar of the download to stderr
        num_classes (int): number of output classes of the model (including the background)
        pretrained_backbone (bool): If True, returns a model with backbone pre-trained on Imagenet
        trainable_backbone_layers (int): number of trainable (not frozen) resnet layers starting from final block.
            Valid values are between 0 and 6, with 6 meaning all backbone layers are trainable.
    """
    weights_name = "fasterrcnn_mobilenet_v3_large_320_fpn_coco"
    defaults = {
        "min_size": 320,
        "max_size": 640,
        "rpn_pre_nms_top_n_test": 150,
        "rpn_post_nms_top_n_test": 150,
        "rpn_score_thresh": 0.05,
    }

    kwargs = {**defaults, **kwargs}
    return _fasterrcnn_mobilenet_v3_large_fpn(weights_name, pretrained=pretrained, progress=progress,
                                              num_classes=num_classes, pretrained_backbone=pretrained_backbone,
                                              trainable_backbone_layers=trainable_backbone_layers, **kwargs)


def fasterrcnn_mobilenet_v3_large_fpn(pretrained=False, progress=True, num_classes=91, pretrained_backbone=True,
                                      trainable_backbone_layers=None, **kwargs):
    """
    Constructs a high resolution Faster R-CNN model with a MobileNetV3-Large FPN backbone.
    It works similarly to Faster R-CNN with ResNet-50 FPN backbone. See
    :func:`~torchvision.models.detection.fasterrcnn_resnet50_fpn` for more
    details.

    Example::

        >>> model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)

    Args:
        pretrained (bool): If True, returns a model pre-trained on COCO train2017
        progress (bool): If True, displays a progress bar of the download to stderr
        num_classes (int): number of output classes of the model (including the background)
        pretrained_backbone (bool): If True, returns a model with backbone pre-trained on Imagenet
        trainable_backbone_layers (int): number of trainable (not frozen) resnet layers starting from final block.
            Valid values are between 0 and 6, with 6 meaning all backbone layers are trainable.
    """
    weights_name = "fasterrcnn_mobilenet_v3_large_fpn_coco"
    defaults = {
        "rpn_score_thresh": 0.05,
    }

    kwargs = {**defaults, **kwargs}
    return _fasterrcnn_mobilenet_v3_large_fpn(weights_name, pretrained=pretrained, progress=progress,
                                              num_classes=num_classes, pretrained_backbone=pretrained_backbone,
                                              trainable_backbone_layers=trainable_backbone_layers, **kwargs)

In [ ]:
from collections import OrderedDict
from typing import Dict, Optional

from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate layers from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Args:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """

    _version = 2
    __annotations__ = {
        "return_layers": Dict[str, str],
    }

    def __init__(self, model: nn.Module, return_layers: Dict[str, str]) -> None:
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")
        orig_return_layers = return_layers
        return_layers = {str(k): str(v) for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super().__init__(layers)
        self.return_layers = orig_return_layers

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.items():
            x = module(x)
            if name in self.return_layers:
                out_name = self.return_layers[name]
                out[out_name] = x
        return out


def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [ ]:

import warnings
from typing import Callable, Dict, List, Optional, Union

from torch import nn, Tensor
from torchvision.ops import misc as misc_nn_ops
from torchvision.ops.feature_pyramid_network import ExtraFPNBlock, FeaturePyramidNetwork, LastLevelMaxPool

from torchvision.models import mobilenet, resnet
#from .._utils import IntermediateLayerGetter


class BackboneWithFPN(nn.Module):
    """
    Adds a FPN on top of a model.
    Internally, it uses torchvision.models._utils.IntermediateLayerGetter to
    extract a submodel that returns the feature maps specified in return_layers.
    The same limitations of IntermediateLayerGetter apply here.
    Args:
        backbone (nn.Module)
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
        in_channels_list (List[int]): number of channels for each feature map
            that is returned, in the order they are present in the OrderedDict
        out_channels (int): number of channels in the FPN.
    Attributes:
        out_channels (int): the number of channels in the FPN
    """

    def __init__(
        self,
        backbone: nn.Module,
        return_layers: Dict[str, str],
        in_channels_list: List[int],
        out_channels: int,
        extra_blocks: Optional[ExtraFPNBlock] = None,
    ) -> None:
        super().__init__()

        if extra_blocks is None:
            extra_blocks = LastLevelMaxPool()

        self.body = IntermediateLayerGetter(backbone, return_layers=return_layers)
        self.fpn = FeaturePyramidNetwork(
            in_channels_list=in_channels_list,
            out_channels=out_channels,
            extra_blocks=extra_blocks,
        )
        self.out_channels = out_channels

    def forward(self, x: Tensor) -> Dict[str, Tensor]:
        x = self.body(x)
        x = self.fpn(x)
        return x


def resnet_fpn_backbone(
    backbone_name: str,
    pretrained: bool,
    norm_layer: Callable[..., nn.Module] = misc_nn_ops.FrozenBatchNorm2d,
    trainable_layers: int = 3,
    returned_layers: Optional[List[int]] = None,
    extra_blocks: Optional[ExtraFPNBlock] = None,
) -> BackboneWithFPN:
    """
    Constructs a specified ResNet backbone with FPN on top. Freezes the specified number of layers in the backbone.
    Examples::
        >>> from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
        >>> backbone = resnet_fpn_backbone('resnet50', pretrained=True, trainable_layers=3)
        >>> # get some dummy image
        >>> x = torch.rand(1,3,64,64)
        >>> # compute the output
        >>> output = backbone(x)
        >>> print([(k, v.shape) for k, v in output.items()])
        >>> # returns
        >>>   [('0', torch.Size([1, 256, 16, 16])),
        >>>    ('1', torch.Size([1, 256, 8, 8])),
        >>>    ('2', torch.Size([1, 256, 4, 4])),
        >>>    ('3', torch.Size([1, 256, 2, 2])),
        >>>    ('pool', torch.Size([1, 256, 1, 1]))]
    Args:
        backbone_name (string): resnet architecture. Possible values are 'resnet18', 'resnet34', 'resnet50',
             'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'wide_resnet50_2', 'wide_resnet101_2'
        pretrained (bool): If True, returns a model with backbone pre-trained on Imagenet
        norm_layer (callable): it is recommended to use the default value. For details visit:
            (https://github.com/facebookresearch/maskrcnn-benchmark/issues/267)
        trainable_layers (int): number of trainable (not frozen) resnet layers starting from final block.
            Valid values are between 0 and 5, with 5 meaning all backbone layers are trainable.
        returned_layers (list of int): The layers of the network to return. Each entry must be in ``[1, 4]``.
            By default all layers are returned.
        extra_blocks (ExtraFPNBlock or None): if provided, extra operations will
            be performed. It is expected to take the fpn features, the original
            features and the names of the original features as input, and returns
            a new list of feature maps and their corresponding names. By
            default a ``LastLevelMaxPool`` is used.
    """
    backbone = resnet.__dict__[backbone_name](pretrained=pretrained, norm_layer=norm_layer)
    return _resnet_fpn_extractor(backbone, trainable_layers, returned_layers, extra_blocks)


def _resnet_fpn_extractor(
    backbone: resnet.ResNet,
    trainable_layers: int,
    returned_layers: Optional[List[int]] = None,
    extra_blocks: Optional[ExtraFPNBlock] = None,
) -> BackboneWithFPN:

    # select layers that wont be frozen
    if trainable_layers < 0 or trainable_layers > 5:
        raise ValueError(f"Trainable layers should be in the range [0,5], got {trainable_layers}")
    layers_to_train = ["layer4", "layer3", "layer2", "layer1", "conv1"][:trainable_layers]
    if trainable_layers == 5:
        layers_to_train.append("bn1")
    for name, parameter in backbone.named_parameters():
        if all([not name.startswith(layer) for layer in layers_to_train]):
            parameter.requires_grad_(False)

    if extra_blocks is None:
        extra_blocks = LastLevelMaxPool()

    if returned_layers is None:
        returned_layers = [1, 2, 3, 4]
    if min(returned_layers) <= 0 or max(returned_layers) >= 5:
        raise ValueError(f"Each returned layer should be in the range [1,4]. Got {returned_layers}")
    return_layers = {f"layer{k}": str(v) for v, k in enumerate(returned_layers)}

    in_channels_stage2 = backbone.inplanes // 8
    in_channels_list = [in_channels_stage2 * 2 ** (i - 1) for i in returned_layers]
    out_channels = 256
    return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels, extra_blocks=extra_blocks)


def _validate_trainable_layers(
    pretrained: bool,
    trainable_backbone_layers: Optional[int],
    max_value: int,
    default_value: int,
) -> int:
    # don't freeze any layers if pretrained model or backbone is not used
    if not pretrained:
        if trainable_backbone_layers is not None:
            warnings.warn(
                "Changing trainable_backbone_layers has not effect if "
                "neither pretrained nor pretrained_backbone have been set to True, "
                f"falling back to trainable_backbone_layers={max_value} so that all layers are trainable"
            )
        trainable_backbone_layers = max_value

    # by default freeze first blocks
    if trainable_backbone_layers is None:
        trainable_backbone_layers = default_value
    if trainable_backbone_layers < 0 or trainable_backbone_layers > max_value:
        raise ValueError(
            f"Trainable backbone layers should be in the range [0,{max_value}], got {trainable_backbone_layers} "
        )
    return trainable_backbone_layers


def mobilenet_backbone(
    backbone_name: str,
    pretrained: bool,
    fpn: bool,
    norm_layer: Callable[..., nn.Module] = misc_nn_ops.FrozenBatchNorm2d,
    trainable_layers: int = 2,
    returned_layers: Optional[List[int]] = None,
    extra_blocks: Optional[ExtraFPNBlock] = None,
) -> nn.Module:
    backbone = mobilenet.__dict__[backbone_name](pretrained=pretrained, norm_layer=norm_layer)
    return _mobilenet_extractor(backbone, fpn, trainable_layers, returned_layers, extra_blocks)


def _mobilenet_extractor(
    backbone: Union[mobilenet.MobileNetV2, mobilenet.MobileNetV3],
    fpn: bool,
    trainable_layers: int,
    returned_layers: Optional[List[int]] = None,
    extra_blocks: Optional[ExtraFPNBlock] = None,
) -> nn.Module:
    backbone = backbone.features
    # Gather the indices of blocks which are strided. These are the locations of C1, ..., Cn-1 blocks.
    # The first and last blocks are always included because they are the C0 (conv1) and Cn.
    stage_indices = [0] + [i for i, b in enumerate(backbone) if getattr(b, "_is_cn", False)] + [len(backbone) - 1]
    num_stages = len(stage_indices)

    # find the index of the layer from which we wont freeze
    if trainable_layers < 0 or trainable_layers > num_stages:
        raise ValueError(f"Trainable layers should be in the range [0,{num_stages}], got {trainable_layers} ")
    freeze_before = len(backbone) if trainable_layers == 0 else stage_indices[num_stages - trainable_layers]

    for b in backbone[:freeze_before]:
        for parameter in b.parameters():
            parameter.requires_grad_(False)

    out_channels = 256
    if fpn:
        if extra_blocks is None:
            extra_blocks = LastLevelMaxPool()

        if returned_layers is None:
            returned_layers = [num_stages - 2, num_stages - 1]
        if min(returned_layers) < 0 or max(returned_layers) >= num_stages:
            raise ValueError(f"Each returned layer should be in the range [0,{num_stages - 1}], got {returned_layers} ")
        return_layers = {f"{stage_indices[k]}": str(v) for v, k in enumerate(returned_layers)}

        in_channels_list = [backbone[stage_indices[i]].out_channels for i in returned_layers]
        return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels, extra_blocks=extra_blocks)
    else:
        m = nn.Sequential(
            backbone,
            # depthwise linear combination of channels to reduce their size
            nn.Conv2d(backbone[-1].out_channels, out_channels, 1),
        )
        m.out_channels = out_channels  # type: ignore[assignment]
        return m

In [ ]:
def get_model_fasterrcnn_resnet50_fpn(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model =fasterrcnn_resnet50_fpn(pretrained = False)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model
def get_model_fasterrcnn_mobilenet_v3_large_320_fpn(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model =fasterrcnn_mobilenet_v3_large_320_fpn(pretrained = False)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model
def get_model_fasterrcnn_mobilenet_v3_large_fpn(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model =fasterrcnn_mobilenet_v3_large_fpn(pretrained = False)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model

# **Extra functions**

In [ ]:

def create_label_txt(path_to_csv):

	data = pd.read_csv(path_to_csv)
	labels = data['class'].unique()

	labels_dict = {}

	# Creat dictionary from array
	for index, label in enumerate(labels):
		labels_dict.__setitem__(index, label)

	# We need to create labels.txt and write labels dictionary into it
	with open('labels.txt', 'w') as f:
		f.write(str(labels_dict))

	return labels_dict	

def parse_one_annot(path, filename, labels_dict):

	data = pd.read_csv(path)

	class_names = data['class'].unique()
	classes_df = data[data["filename"] == filename]["class"]
	classes_array = classes_df.to_numpy()
	
	boxes_df = data[data["filename"] == filename][["xmin", "ymin", "xmax", "ymax"]]
	boxes_array = boxes_df.to_numpy()
	
	classes = []
	for key, value in labels_dict.items():
		for i in classes_array:
			if i == value:
				classes.append(key)

	# Convert list to tuple
	classes = tuple(classes)

	return boxes_array, classes

def get_transforms(train):

	transforms = []

	# Convert numpy image to PyTorch Tensor
	transforms.append(T.ToTensor())

	if train:
		# Data augmentation
		transforms.append(T.RandomHorizontalFlip(0.5))

	return T.Compose(transforms)

In [ ]:
def iou(a,b):
    ulresult=0
    count=0
    count1=0
    ones=[0 for i in range(len(a))]
    #print(len(a),"     ",len(b))
    for f in range(len(a)):
        i=a[f]
        for j in b:
            xmin,ymin,xmax,ymax=0,0,0,0
            if(i[0]<j[0] and i[1]<j[1] and i[2]>j[0] and i[3]>j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ones[f]=1
                    ulresult+=result
                    count+=1
            elif(i[1]>j[1] and i[1]<j[3] and i[2]>j[0]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ones[f]=1
                    ulresult+=result
                    count+=1
            elif(i[0]>j[0] and i[0]<j[2] and i[3]<j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ones[f]=1
                    ulresult+=result
                    count+=1
            elif(i[0]>j[0] and i[1]>j[1] and i[0]<j[2] and i[1]<j[3]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ones[f]=1
                    ulresult+=result
                    count+=1
    for i in range(len(ones)):
      if(ones[i]==0):
        count1+=1
    if(count==0):
        return(-1)
    else:
        print(ulresult,"      ",count,"    ",count1,"   ",len(ones))
        return(ulresult/(count+count1))
import cv2
def zaccard(data_loader_test,loaded_model):
    ulresult=0
    ulcount=0
    zzz=0
    #print(len(dataloaders_test))
    for i_batch, sample_batched in enumerate(data_loader_test):
        img,targets=sample_batched
        #print("For batch:",i_batch)
         #print(targets)
        zzz+=1
    #image = cv2.imread(img_path)
 #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 # img = torchvision.transforms.ToTensor()(img)
    #print(img)
        img=img[0]
    #img = torchvision.transforms.ToTensor()(img)
        #loaded_model = get_model(6)
        #loaded_model.load_state_dict(torch.load(output_dir + "/fasterRCNNfinalmodel"))
        img=img.to(device=device)
        #targets=targets.to(device=device)
        loaded_model.eval()
        with torch.no_grad():
            prediction = loaded_model([img])
            boxes=prediction[0]["boxes"]
       # print("boxes:",boxes)
    #print(targets[0]["boxes"])
        targets=np.array(targets[0]["boxes"])
        boxes=boxes.cpu()
        boxes=np.array(boxes)
    #print("targets",targets)
    #print("boxes",boxes)
        result=iou(targets,boxes)
        print(result)
        if(result!=-1):
            ulresult+=result
            ulcount+=1

    count1=0
    print(count1)
    """for i in range(len(ones)):
      if(ones[i]==0):
        count1+=1"""
    if(ulcount==0):
        
        return(0)
    return(ulresult/ulcount)


In [ ]:
def priority_iou(a):

    for m in range(len(a)-1,-1,-1):
        #print(m)
        for n in range(m):
            i=a[m]
            j=a[n]
            #print(i,"    ",j)
            #return(None)
            xmin,ymin,xmax,ymax=0,0,0,0
            if(i[0]<j[0] and i[1]<j[1] and i[2]>j[0] and i[3]>j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=max(area/(aarea+barea-area),area/aarea,area/barea)
                if(result>0.5):
                    a[m]=1
                    break
  
            elif(i[1]>j[1] and i[1]<j[3] and i[2]>j[0]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=max(area/(aarea+barea-area),area/aarea,area/barea)
                if(result>0.5):
                    a[m]=1
                    break
                    
            elif(i[0]>j[0] and i[0]<j[2] and i[3]<j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=max(area/(aarea+barea-area),area/aarea,area/barea)
                if(result>0.5):
                    a[m]=1
                    break
            elif(i[0]>j[0] and i[1]>j[1] and i[0]<j[2] and i[1]<j[3]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=max(area/(aarea+barea-area),area/aarea,area/barea)
                if(result>0.5):
                    a[m]=1
                    break
    #print(a)
    b=[]
    for i in a:
      if(i!=1):
        b.append(i)
    return(b)

# **Dataset**

In [ ]:
class ListDataset(Dataset):
    def __init__(self, img_files,label_files, img_size=448,transforms = None):
        self.transforms = transforms
        self.inputs_dtype = torch.float64
        self.targets_dtype = torch.long
        self.img_files=img_files
        self.label_files=label_files
        self.img_shape = (img_size, img_size)
        self.max_objects = 500

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):

        img_path = self.img_files[index % len(self.img_files)].rstrip()
        img = cv2.imread(img_path)
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        label_path = self.label_files[index % len(self.img_files)].rstrip()
        labels = None
        labelsz=[]
        #print("input image shape",input_img.shape)
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path)
            cls=[]
            for i in range(len(labels)):
                if(labels[i][3]>0.0001 and labels[i][4]>0.0001):
                    labelsz.append(labels[i][1:])
                    cls.append(labels[i][0])
            labels=labelsz
            labels=np.array(labels)
            labels.reshape(-1, 4)
            x1 = (labels[:,0])*input_img.shape[1]#w * (labels[:, 1] - labels[:, 3]/2)
            y1 = (labels[:,1])*input_img.shape[0]#h * (labels[:, 2] - labels[:, 4]/2)
            x2 = (labels[:,0]+labels[:,2])*input_img.shape[1]#w * (labels[:, 1] + labels[:, 3]/2)
            y2 = (labels[:,1]+labels[:,3])*input_img.shape[0]#h * (labels[:, 2] + labels[:, 4]/2)
            # Calculate ratios from coordinates
            labels[:, 0] = x1 # / padded_w
            labels[:, 1] = y1# / padded_w
            labels[:, 2] = x2# / padded_h
            labels[:, 3] = y2# / padded_h
        labels = np.where(labels<0, 0, labels)
        obj=[]
        iscorwd=[]
        obj=cls
        """for i in range(len(labels)):
         
          #l.append(labels)
          #labelsx.append(labels[i][:].tolist())
          obj.append(1)
          #iscrowd.append(1)"""
        labelsx=np.array(labels)
        
        num_box=len(labels)
        # Fill matrix
        filled_labels = np.zeros((self.max_objects, 4))
        if labels is not None:
            filled_labels[range(len(labels))[:self.max_objects]] = labels[:self.max_objects]
        iscrowd = torch.zeros((num_box,), dtype=torch.int64)
        #img =img.reshape(3,1536,2048)
        labels=labelsx
        labels=labels.tolist()
        #obj=obj.tolist()
        #print(labels)
        #print(obj)
        labels=np.array(labels)
        
        obj=np.array(obj)
        #labels=labels.tolist()
        #obj=obj.tolist()
        img=img/255
              
        


        boxes = torch.as_tensor(labels, dtype = torch.float32)
        
        obj=[1.0 for i in range(len(obj))]
        #print("obj",obj)
        labels = torch.tensor(obj, dtype=torch.int64)
        #boxes*=255
        image_id = torch.tensor([index])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        iscrowd = torch.tensor(obj, dtype=torch.int64)
        
        #img=img.astype(double)
        target = {}
        #print("labels",labels)
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        #print("start list target") 
        #print(target)
        #print("end list target")
       # print(boxes)
        if self.transforms is not None:
          input_img, target = self.transforms(input_img, target)
        #print("these are boxes:",boxes)
        #print(labels)
        #print(image_id)
        #print(area)
        #print(iscrowd)
        #a=list(targets)
        #a.append(target)
        #print("image",type(input_img),"boxes:",type(boxes),",labels:",type(labels),",image id:",image_id,",area:",area,",iscrowd:",iscrowd)
        #print("targets:",target)
        #print("image:",input_img)
        return input_img,target


In [ ]:
batch_size = 1
inp_dim = 800
path='/content/gdrive/MyDrive/Data_localization-FNAC/'
image_train=glob.glob(path+'train/images/'+'*.jpg')
image_val=glob.glob(path+'val/images/'+'*.jpg')
label_files = []
inputs_dtype = torch.float32
targets_dtype = torch.long
label_train=[]
label_val=[]
for img in image_train:
    label_train.append(path+'train/text/'+img[len(path)+len('train/images/')-1:-4]+'.txt')
for img in image_val:
    label_val.append(path+'val/text/'+img[len(path)+len('val/images/')-1:-4]+'.txt')
print(len(image_val))
print(len(label_val))
train_dataset=ListDataset(image_train,label_train, img_size=inp_dim, transforms = get_transforms(train = True))
valid_dataset=ListDataset(image_val,label_val, img_size=inp_dim, transforms = get_transforms(train = False))
dataloaders_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=1,collate_fn = utils.collate_fn)
dataloaders_val = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True,num_workers=1,collate_fn = utils.collate_fn)
print(len(dataloaders_train))
for i_batch, sample in enumerate(dataloaders_train):
    #print(type(sample))
    img,targets=sample
    #print("image")
    #print(img)
    print(type(img[0]))
    #print("start target")
    print(targets)
    boxes=targets[0]["boxes"]
    """for element in boxes:
      start_point=(element[0],element[1])
      end_point=(element[2],element[3])
      cv2.rectangle(img[0],start_point,end_point,(255,0,0),3)
      cv2_imshow(input_img)"""
    #cv2_imshow(img[0])  
    #print("end target")
    if i_batch == 0:
        break

12
12
49
<class 'torch.Tensor'>
({'boxes': tensor([[1197.0010,   76.9997, 1255.0004,  136.9997],
        [ 808.0000,  430.0003,  860.0000,  490.0003],
        [ 930.0006,  289.9997,  998.0006,  345.9994],
        [1057.9994,  454.0003, 1125.9994,  504.0000],
        [ 713.9994,  618.0000,  776.0000,  697.9997],
        [ 649.9994,  364.0003,  704.0000,  414.0000],
        [ 594.0006,  420.0000,  644.0000,  466.0003],
        [ 349.9994,  553.0003,  397.9994,  622.0003],
        [ 344.0013,  382.0003,  382.0006,  422.0006],
        [ 258.0006,  328.0003,  322.0006,  390.0000],
        [ 216.0000,  804.0000,  242.0006,  829.9997],
        [   4.9996,  562.0003,   28.0000,  584.0006],
        [  23.0004,  435.0000,  122.0006,  598.0003],
        [  24.0000,  538.0003,   68.9996,  572.0006],
        [  15.0004,  405.0000,   71.0004,  453.0000],
        [  36.0000,  442.0003,   76.9996,  470.0006],
        [   0.0000,  468.0000,   48.0000,  553.0003],
        [ 328.9997,  774.0000,  428.000

# **Training Resnet 50**

In [ ]:
test_set_length = 40 		 # Test set (number of images)
train_batch_size = 1  		 # Train batch size
test_batch_size = 1    		 # Test batch size
num_classes = 2	 # Number of classes
learning_rate = 0.005  		 # Learning rate
num_epochs=10
eps=0
device='cuda'
model =get_model_fasterrcnn_resnet50_fpn(num_classes)
model.to(device = device)
modelfinal=get_model_fasterrcnn_resnet50_fpn(num_classes)
modelfinal.to(device = device)
zzzfinal=0
# Construct the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = learning_rate, momentum = 0.9, weight_decay = 0.0005)
output_path='/content/gdrive/MyDrive/'
# Learning rate scheduler decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
for epoch in range(num_epochs):   
    
    train_one_epoch(model, optimizer, dataloaders_train, device, epoch, print_freq = 10)
    lr_scheduler.step()
    
    zzz=zaccard(dataloaders_val,model)
    print("Zaccard index for epoch",epoch,"is",zzz)
    if(zzzfinal<zzz):
        modelfinal=model
        zzzfinal=zzz
        eps=epoch

# Save the model state	
#torch.save(modelfinal.state_dict(), output_dir + "/fasterRCNNultimate")

torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac_fasterrcnn_resnet50_fpn.pt")

hello
hello
Epoch: [0]  [ 0/49]  eta: 0:01:28  lr: 0.000109  loss: 1.6210 (1.6210)  loss_classifier: 0.6253 (0.6253)  loss_box_reg: 0.0783 (0.0783)  loss_objectness: 0.6925 (0.6925)  loss_rpn_box_reg: 0.2249 (0.2249)  time: 1.8094  data: 0.6527  max mem: 1397
Epoch: [0]  [10/49]  eta: 0:00:23  lr: 0.001150  loss: 1.5643 (1.6103)  loss_classifier: 0.6184 (0.5932)  loss_box_reg: 0.0783 (0.1476)  loss_objectness: 0.6925 (0.6918)  loss_rpn_box_reg: 0.1583 (0.1776)  time: 0.5971  data: 0.2315  max mem: 5459
Epoch: [0]  [20/49]  eta: 0:00:16  lr: 0.002190  loss: 1.6628 (1.6426)  loss_classifier: 0.5617 (0.5443)  loss_box_reg: 0.2596 (0.2633)  loss_objectness: 0.6862 (0.6774)  loss_rpn_box_reg: 0.1436 (0.1577)  time: 0.4919  data: 0.2051  max mem: 5459
Epoch: [0]  [30/49]  eta: 0:00:10  lr: 0.003231  loss: 1.8927 (1.7672)  loss_classifier: 0.5205 (0.5442)  loss_box_reg: 0.5685 (0.4073)  loss_objectness: 0.6527 (0.6566)  loss_rpn_box_reg: 0.1470 (0.1591)  time: 0.4979  data: 0.2133  max mem: 5

In [ ]:
torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac_resnet_300bb_1.pt")

# **Training MobileNet 320**

In [ ]:
test_set_length = 40 		 # Test set (number of images)
train_batch_size = 1  		 # Train batch size
test_batch_size = 1    		 # Test batch size
num_classes = 2	 # Number of classes
learning_rate = 0.005  		 # Learning rate
num_epochs=10
eps=0
device='cuda'
model = get_model_fasterrcnn_mobilenet_v3_large_320_fpn(num_classes)
model.to(device = device)
modelfinal=get_model_fasterrcnn_mobilenet_v3_large_320_fpn(num_classes)
modelfinal.to(device = device)
zzzfinal=0
# Construct the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = learning_rate, momentum = 0.9, weight_decay = 0.0005)
output_path='/content/gdrive/MyDrive/'
# Learning rate scheduler decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
for epoch in range(num_epochs):   
    
    train_one_epoch(model, optimizer, dataloaders_train, device, epoch, print_freq = 10)
    lr_scheduler.step()
    
    zzz=zaccard(dataloaders_val,model)
    print("Zaccard index for epoch",epoch,"is",zzz)
    if(zzzfinal<zzz):
        modelfinal=model
        zzzfinal=zzz
        eps=epoch

# Save the model state	
#torch.save(modelfinal.state_dict(), output_dir + "/fasterRCNNultimate")

torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac_fasterrcnn_mobilenet_v3_large_320_fpn.pt")

hello
hello
Epoch: [0]  [ 0/49]  eta: 0:00:13  lr: 0.000109  loss: 2.1381 (2.1381)  loss_classifier: 0.7133 (0.7133)  loss_box_reg: 0.2102 (0.2102)  loss_objectness: 0.6986 (0.6986)  loss_rpn_box_reg: 0.5160 (0.5160)  time: 0.2661  data: 0.1629  max mem: 5459
Epoch: [0]  [10/49]  eta: 0:00:03  lr: 0.001150  loss: 2.1007 (2.0056)  loss_classifier: 0.6787 (0.6456)  loss_box_reg: 0.0648 (0.0748)  loss_objectness: 0.6968 (0.6968)  loss_rpn_box_reg: 0.6974 (0.5884)  time: 0.1021  data: 0.0224  max mem: 5459
Epoch: [0]  [20/49]  eta: 0:00:02  lr: 0.002190  loss: 2.0533 (1.9315)  loss_classifier: 0.6053 (0.5760)  loss_box_reg: 0.0392 (0.0895)  loss_objectness: 0.6938 (0.6913)  loss_rpn_box_reg: 0.6255 (0.5747)  time: 0.0826  data: 0.0079  max mem: 5459
Epoch: [0]  [30/49]  eta: 0:00:01  lr: 0.003231  loss: 1.8470 (1.8383)  loss_classifier: 0.5198 (0.5404)  loss_box_reg: 0.0516 (0.1009)  loss_objectness: 0.6763 (0.6811)  loss_rpn_box_reg: 0.4598 (0.5158)  time: 0.0804  data: 0.0082  max mem: 5

In [ ]:
torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac_resnet_300bb_1.pt")

# **Training MobileNet**

In [ ]:
test_set_length = 40 		 # Test set (number of images)
train_batch_size = 1  		 # Train batch size
test_batch_size = 1    		 # Test batch size
num_classes = 2	 # Number of classes
learning_rate = 0.005  		 # Learning rate
num_epochs=20
eps=0
device='cuda'
model = get_model_fasterrcnn_mobilenet_v3_large_fpn(num_classes)
model.to(device = device)
modelfinal=get_model_fasterrcnn_mobilenet_v3_large_fpn(num_classes)
modelfinal.to(device = device)
zzzfinal=0
# Construct the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = learning_rate, momentum = 0.9, weight_decay = 0.0005)
output_path='/content/gdrive/MyDrive/'
# Learning rate scheduler decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
for epoch in range(num_epochs):   
    
    train_one_epoch(model, optimizer, dataloaders_train, device, epoch, print_freq = 10)
    lr_scheduler.step()
    
    zzz=zaccard(dataloaders_val,model)
    print("Zaccard index for epoch",epoch,"is",zzz)
    if(zzzfinal<zzz):
        modelfinal=model
        zzzfinal=zzz
        eps=epoch

# Save the model state	
#torch.save(modelfinal.state_dict(), output_dir + "/fasterRCNNultimate")

torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac__mobilenet_v3_large_fpn.pt")

hello
hello
Epoch: [0]  [ 0/49]  eta: 0:00:15  lr: 0.000109  loss: 1.9300 (1.9300)  loss_classifier: 0.7226 (0.7226)  loss_box_reg: 0.1355 (0.1355)  loss_objectness: 0.6938 (0.6938)  loss_rpn_box_reg: 0.3781 (0.3781)  time: 0.3074  data: 0.1682  max mem: 1642
Epoch: [0]  [10/49]  eta: 0:00:04  lr: 0.001150  loss: 1.9079 (1.8537)  loss_classifier: 0.6837 (0.6574)  loss_box_reg: 0.1355 (0.1524)  loss_objectness: 0.6915 (0.6913)  loss_rpn_box_reg: 0.3781 (0.3526)  time: 0.1140  data: 0.0219  max mem: 1646
Epoch: [0]  [20/49]  eta: 0:00:02  lr: 0.002190  loss: 1.8113 (1.7688)  loss_classifier: 0.6037 (0.5896)  loss_box_reg: 0.1380 (0.1942)  loss_objectness: 0.6891 (0.6885)  loss_rpn_box_reg: 0.3191 (0.2965)  time: 0.0922  data: 0.0075  max mem: 1806
Epoch: [0]  [30/49]  eta: 0:00:01  lr: 0.003231  loss: 1.7724 (1.7857)  loss_classifier: 0.5310 (0.5675)  loss_box_reg: 0.2896 (0.2445)  loss_objectness: 0.6794 (0.6813)  loss_rpn_box_reg: 0.2761 (0.2924)  time: 0.0907  data: 0.0074  max mem: 1

In [ ]:
torch.save(modelfinal.state_dict(), output_path + "fasterrcnn_fnac_resnet_300bb_1.pt")

# **Testing**

In [ ]:
#import torch
#import numpy as np
#import cv2
##import os
#import pandas as pd
#import torchvision
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#import torch
from torchvision.utils import save_image

"""def get_model(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model"""
input_path = "/content/gdrive/MyDrive/Data_localization-FNAC/val/images/"
output_path="/content/gdrive/MyDrive/new/"
model_path="/content/gdrive/MyDrive/fasterrcnn_fnac_resnet_300bb_1.pt"
model = get_model(2)
model.load_state_dict(torch.load(os.path.join(model_path), map_location = 'cpu'))
model=model.eval().cuda()
image_array=os.listdir(input_path)
for detect in range(len(image_array)):
  image = cv2.imread(input_path+image_array[detect])
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img = torchvision.transforms.ToTensor()(img)
  img=img.cuda()
  #model = get_model(6)
  #model.load_state_dict(torch.load(os.path.join(model_path), map_location = 'cpu'))

  #model.eval()
  with torch.no_grad():
	  #print(prediction)
		
	  prediction = model([img])
	  
	  boxes=prediction[0]["boxes"]
	  scores=prediction[0]["scores"]
	  #print(scores)
  #print(type(boxes))
  #boxes=boxes.tolist()
  boxes=priority_iou(boxes.tolist())
  boxes=torch.from_numpy(np.array(boxes))
  #print(boxes[0][0].dtype)
  print(boxes.shape)
 
  for element in range(len(boxes)):
    if(scores[element]>0):
	      start_point=(boxes[element][0],boxes[element][1])
	      end_point=(boxes[element][2],boxes[element][3])
	      cv2.rectangle(image,start_point,end_point,(255,0,0),3)
  #break
  cv2.imwrite(output_path+image_array[detect], image)
  print(detect,"             ",image_array[detect])
  """for element in boxes:
	  start_point=(element[0],element[1])
	  end_point=(element[2],element[3])
	  cv2.rectangle(image,start_point,end_point,(255,0,0),3)
  cv2.imwrite(output_path+image_array[detect], image)
  print(detect,"             ",image_array[detect])"""
  
  #break

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

hello
torch.Size([152, 4])
0               B10D.jpg
torch.Size([90, 4])
1               B11A.jpg
torch.Size([159, 4])
2               B20F.jpg
torch.Size([123, 4])
3               B14A.jpg
torch.Size([77, 4])
4               B19.jpg
torch.Size([98, 4])
5               Bb32e.jpg
torch.Size([177, 4])
6               M51B.jpg
torch.Size([109, 4])
7               M53I.jpg
torch.Size([94, 4])
8               M52E.jpg
torch.Size([111, 4])
9               M52I.jpg
torch.Size([92, 4])
10               M52C.jpg
torch.Size([120, 4])
11               M53B.jpg


In [ ]:
print(prediction)

[{'boxes': tensor([[ 808.8229,  692.1614,  862.0474,  743.3681],
        [ 420.4780,  570.8420,  472.0190,  622.3196],
        [1069.9417,  436.8656, 1139.3029,  501.0830],
        [ 786.1982,  244.3059,  854.1392,  310.2119],
        [ 430.2097,  275.0708,  481.6370,  325.3628],
        [ 214.3897,  410.5661,  265.7202,  462.2716],
        [1007.4999,  362.7847, 1059.1975,  412.6015],
        [ 532.2956,  262.3110,  584.1590,  313.8087],
        [ 310.0874,  276.8058,  360.3513,  326.9963],
        [ 730.0942,  276.6353,  783.4250,  329.6028],
        [ 216.8819,  466.2226,  279.9054,  524.6863],
        [ 918.3499,  269.8150,  969.1335,  319.6887],
        [ 452.7331,  433.8010,  518.1171,  496.5770],
        [1051.4884,  324.2272, 1117.4420,  389.1034],
        [ 365.0124,  444.9696,  420.2438,  500.4689],
        [ 499.7889,  306.6310,  552.9693,  356.3370],
        [ 136.7482,  508.1434,  204.1783,  569.3141],
        [ 580.6343,  347.3147,  630.9330,  395.8591],
        [ 233.268

In [ ]:
print(len(boxes.tolist()))
boxes1=priority_iou(boxes.tolist())
print(len(boxes1))

200
95


# **Calculating IOU**

In [ ]:
#for creating ground truth
input_path = "/content/gdrive/MyDrive/Data_localization-FNAC/val/images/"
output_path="/content/gdrive/MyDrive/new3/"
#model_path="/content/gdrive/MyDrive/fasterrcnnfnac.pt"
#image_array=os.listdir(input_path)
for detect in range(len(image_array)):
  
  image = cv2.imread(input_path+image_array[detect])
  frame=image
  labels_path="/content/gdrive/MyDrive/Data_localization-FNAC/val/text/"+image_array[detect][:-3]+"txt"
  #print(labels_path)
  #print(labels_path)
  labels = np.loadtxt(labels_path)
  #labels=labels.tolist()
  #print(labels)
  labels=labels.tolist()
  b=[]
  for i in range(len(labels)):
    a=[]
    a.append(labels[i][1]*frame.shape[1])
    a.append(labels[i][2]*frame.shape[0])
    a.append((labels[i][1]+labels[i][3])*frame.shape[1])
    a.append((labels[i][2]+labels[i][4])*frame.shape[0])
    b.append(a)
  labels=b
#print(b)
  #print(labels)
  #print(len(labels))
  for element in labels:
	  start_point=(int(element[0]),int(element[1]))
	  end_point=(int(element[2]),int(element[3]))
	  #print(type(element[0]))
	  #print(type(image),"   ",type(start_point),"    ",type(end_point))
	  cv2.rectangle(image,start_point,end_point,(255,255,0),3)
  """for element in boxes:
	  start_point=(element[0],element[1])
	  end_point=(element[2],element[3])
	  cv2.rectangle(image,start_point,end_point,(255,0,0),3)"""
  cv2.imwrite(output_path+image_array[detect], image)
  print(detect,"             ",image_array[detect])
  #break

FOR IMAGE : B10D.jpg
0               B10D.jpg
FOR IMAGE : B11A.jpg
1               B11A.jpg
FOR IMAGE : B20F.jpg
2               B20F.jpg
FOR IMAGE : B14A.jpg
3               B14A.jpg
FOR IMAGE : B19.jpg
4               B19.jpg
FOR IMAGE : Bb32e.jpg
5               Bb32e.jpg
FOR IMAGE : M51B.jpg
6               M51B.jpg
FOR IMAGE : M53I.jpg
7               M53I.jpg
FOR IMAGE : M52E.jpg
8               M52E.jpg
FOR IMAGE : M52I.jpg
9               M52I.jpg
FOR IMAGE : M52C.jpg
10               M52C.jpg
FOR IMAGE : M53B.jpg
11               M53B.jpg


In [ ]:
def iou(a,b):
    # a - labels/ground_truth
    # b - boxes/output
    ulresult=0
    #count=0
    true_positive=0
    false_negative=0
    false_positive=0
    ones=[0 for i in range(len(a))]
    twos=[0 for i in range(len(b))]
    #print(len(a),"     ",len(b))
    for f in range(len(a)):
        i=a[f]
        for g in range(len(b)):
            j=b[g]
            xmin,ymin,xmax,ymax=0,0,0,0
            if(i[0]<j[0] and i[1]<j[1] and i[2]>j[0] and i[3]>j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                #result=area/(aarea+barea-area)
                result=area/aarea
                if(result>0.5 and not ones[f]):
                    ones[f]=1
                    twos[g]=1
                    ulresult+=result
            elif(i[1]>j[1] and i[1]<j[3] and i[2]>j[0]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                #result=area/(aarea+barea-area)
                result=area/aarea
                if(result>0.5 and not ones[f]):
                    ones[f]=1
                    twos[g]=1
                    ulresult+=result
            elif(i[0]>j[0] and i[0]<j[2] and i[3]<j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                #result=area/(aarea+barea-area)
                result=area/aarea
                if(result>0.5 and not ones[f]):
                    ones[f]=1
                    twos[g]=1
                    ulresult+=result
            elif(i[0]>j[0] and i[1]>j[1] and i[0]<j[2] and i[1]<j[3]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                #result=area/(aarea+barea-area)
                result=area/aarea
                if(result>0.5 and not ones[f]):
                    ones[f]=1
                    twos[g]=1
                    ulresult+=result
    for i in range(len(ones)):
      if(ones[i]==0):
        false_negative+=1
      else:
        true_positive+=1
    for i in range(len(twos)):
       if(twos[i]==0):
         false_positive+=1
    if(true_positive==0):
        return(-1)
    else:
        print("TOTAL OVERLAPPING:",ulresult)
        print("TOTAL DETECTIONS:",len(b))
        print("TOTAL LABELS: ",len(a))
        print("TRUE POSITIVE: ",true_positive)
        print("FALSE NEGATIVE:",false_negative)
        print("FALSE POSITIVE",false_positive)
        #print(ulresult,"      ",count,"    ",count1,"    ",len(ones))
        #return(ulresult/(count))
        return(ulresult/true_positive,true_positive/len(a))
        #return(ulresult/true_positive,ulresult/(true_positive+false_negative),true_positive/(true_positive+false_negative))
import cv2
def zaccard(data_loader_test,loaded_model):
    ulresult=1
    ulcount=0
    zzz=0
    iou=0
    iou_total=0
    tf=0
    #print(len(dataloaders_test))
    for i_batch, sample_batched in enumerate(data_loader_test):
        img,targets=sample_batched
        #print("For batch:",i_batch)
         #print(targets)
        zzz+=1
    #image = cv2.imread(img_path)
 #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 # img = torchvision.transforms.ToTensor()(img)
    #print(img)
        img=img[0]
    #img = torchvision.transforms.ToTensor()(img)
        #loaded_model = get_model(6)
        #loaded_model.load_state_dict(torch.load(output_dir + "/fasterRCNNfinalmodel"))
        img=img.to(device=device)
        #targets=targets.to(device=device)
        loaded_model.eval()
        with torch.no_grad():
            prediction = loaded_model([img])
            boxes=prediction[0]["boxes"]
       # print("boxes:",boxes)
    #print(targets[0]["boxes"])
        targets=np.array(targets[0]["boxes"])
        boxes=boxes.cpu()
        boxes=np.array(boxes)
    #print("targets",targets)
    #print("boxes",boxes)
        result1,result2,result3=iou(targets,boxes)
        #print(result)
        if(result1!=-1):
            iou+=result1
            iou_total+=result2
            tf+=result3
            ulcount+=1

    count1=0
    print(count1)
    """for i in range(len(ones)):
      if(ones[i]==0):
        count1+=1"""
    if(ulcount==0):
        return(0)
    print("iou:  ",iou,", iou total:  ",iou_total,", true positive:  ",tf)    
    return(ulresult/ulcount)


In [ ]:
def iou(a,b):
    # a - labels/ground_truth
    # b - boxes/output
    ulresult=0
    #count=0
    true_positive=0
    false_negative=0
    false_positive=0
    ones=[0 for i in range(len(a))]
    twos=[0 for i in range(len(b))]
    #print(len(a),"     ",len(b))
    for f in range(len(a)):
        boxA=a[f]
        for g in range(len(b)):
            boxB=b[g]
            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])
	          # compute the area of intersection rectangle
            interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	          # compute the area of both the prediction and ground-truth
	          # rectangles
            boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
            boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	          # compute the intersection over union by taking the intersection
	          #     area and dividing it by the sum of prediction + ground-truth
	          # areas - the interesection area
            result = interArea / float(boxAArea + boxBArea - interArea)
	          # return the intersection over union value
            #return iou
            #result=area/aarea
            if(result>0.5):
              ones[f]=1
              twos[g]=1
              ulresult+=result
    for i in range(len(ones)):
      if(ones[i]==0):
        false_negative+=1
      else:
        true_positive+=1
    for i in range(len(twos)):
       if(twos[i]==0):
         false_positive+=1
    if(true_positive==0):
        return(-1)
    else:
        print("TOTAL OVERLAPPING:",ulresult)
        print("TOTAL DETECTIONS:",len(b))
        print("TOTAL LABELS: ",len(a))
        print("TRUE POSITIVE: ",true_positive)
        print("FALSE NEGATIVE:",false_negative)
        print("FALSE POSITIVE",false_positive)
        #print(ulresult,"      ",count,"    ",count1,"    ",len(ones))
        #return(ulresult/(count))
        return(ulresult/true_positive,true_positive/len(a))
        #return(ulresult/true_positive,ulresult/(true_positive+false_negative),true_positive/(true_positive+false_negative))

def bb_intersection_over_union(A, B):
  #A-labels
  #B-predictions
	# determine the (x, y)-coordinates of the intersection rectangle
  for i in A:
    for j in B:
	    xA = max(boxA[0], boxB[0])
	    yA = max(boxA[1], boxB[1])
	    xB = min(boxA[2], boxB[2])
	    yB = min(boxA[3], boxB[3])
	    # compute the area of intersection rectangle
	    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	    # compute the area of both the prediction and ground-truth
	    # rectangles
	    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	    # compute the intersection over union by taking the intersection
	    #     area and dividing it by the sum of prediction + ground-truth
	    # areas - the interesection area
	    iou = interArea / float(boxAArea + boxBArea - interArea)
	    # return the intersection over union value
	    return iou

In [ ]:
def priority_iou(a):

    for m in range(len(a)-1,-1,-1):
        #print(m)
        for n in range(m):

            boxA=a[m]
            
            boxB=a[n]
            #print(boxA,"    ",boxB)
            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])
	          # compute the area of intersection rectangle
            interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	          # compute the area of both the prediction and ground-truth
	          # rectangles
            boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
            boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	          # compute the intersection over union by taking the intersection
	          #     area and dividing it by the sum of prediction + ground-truth
	          # areas - the interesection area
            result = interArea / float(boxAArea + boxBArea - interArea)
	          # return the intersection over union value
            #return iou
            #result=area/aarea
            if(result>0.5):
              a[m]=1
              break
    #print(a)
    b=[]
    for i in a:
      if(i!=1):
        b.append(i)
    return(b)

In [ ]:
from torchvision.utils import save_image
from google.colab.patches import cv2_imshow
input_path = "/content/gdrive/MyDrive/Data_localization-FNAC/val/images/"
output_path="/content/gdrive/MyDrive/mobilenet/"
model_path_resent_50="/content/gdrive/MyDrive/fasterrcnn_fnac_fasterrcnn_resnet50_fpn.pt"
model_path_mobilenet='/content/gdrive/MyDrive/fasterrcnn_fnac__mobilenet_v3_large_fpn.pt'
model_path_mobilenet_large_320='/content/gdrive/MyDrive/fasterrcnn_fnac_fasterrcnn_mobilenet_v3_large_320_fpn.pt'
model_resnet_50 = get_model_fasterrcnn_resnet50_fpn(2)
model_resnet_50.load_state_dict(torch.load(os.path.join(model_path_resent_50), map_location = 'cpu'))
model_resnet_50=model_resnet_50.eval().cuda()
model_mobilenet = get_model_fasterrcnn_mobilenet_v3_large_fpn(2)
model_mobilenet.load_state_dict(torch.load(os.path.join(model_path_mobilenet), map_location = 'cpu'))
model_mobilenet=model_mobilenet.eval().cuda()
model_mobilenet_320 = get_model_fasterrcnn_mobilenet_v3_large_320_fpn(2)
model_mobilenet_320.load_state_dict(torch.load(os.path.join(model_path_mobilenet_large_320), map_location = 'cpu'))
model_mobilenet_320=model_mobilenet_320.eval().cuda()
image_array=os.listdir(input_path)
print(image_array)
area_result_resnet_50=[]
area_result_mobilenet=[]
area_result_mobilenet_320=[]
area_result=[]
accuracy_result_resnet_50=[]
accuracy_result_mobilenet=[]
accuracy_result_mobilenet_320=[]
accuracy_result=[]
for detect in range(len(image_array)):
  print("FOR IMAGE :",image_array[detect])
  image = cv2.imread(input_path+image_array[detect])
  frame=image
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img = torchvision.transforms.ToTensor()(img)
  img=img.cuda()

  labels_path="/content/gdrive/MyDrive/Data_localization-FNAC/val/text/"+image_array[detect][:-3]+"txt"
  #print(labels_path)
  #print(labels_path)
  labels = np.loadtxt(labels_path)
  #labels=labels.tolist()
  #print(labels)
  labels=labels.tolist()
  b=[]
  for i in range(len(labels)):
    a=[]
    a.append(labels[i][1]*frame.shape[1])
    a.append(labels[i][2]*frame.shape[0])
    a.append((labels[i][1]+labels[i][3])*frame.shape[1])
    a.append((labels[i][2]+labels[i][4])*frame.shape[0])
    b.append(a)
  labels=b
#print(b)
  #print(labels)
  #print(len(labels))
  for element in labels:
	  start_point=(int(element[0]),int(element[1]))
	  end_point=(int(element[2]),int(element[3]))
	  #print(type(element[0]))
	  #print(type(image),"   ",type(start_point),"    ",type(end_point))
	  cv2.rectangle(image,start_point,end_point,(255,255,0),3)

  with torch.no_grad():
	  prediction =model_resnet_50([img])
	  boxes_resnet=prediction[0]["boxes"]
	  scores_resnet=prediction[0]["scores"]
  print("FOR RESNET")
  boxes_resnet=priority_iou(boxes_resnet.tolist())
  boxes_resnet=torch.from_numpy(np.array(boxes_resnet))
  result_resnet_50=iou(labels,boxes_resnet)
  area_result_resnet_50.append(result_resnet_50[0])
  accuracy_result_resnet_50.append(result_resnet_50[1])
  with torch.no_grad():
	  prediction =model_mobilenet([img])
	  boxes_mobilenet=prediction[0]["boxes"]
	  scores_mobilenet=prediction[0]["scores"]
  print("FOR MOBILENET")
  #print(len(boxes_mobilenet))
  print(boxes_mobilenet.tolist())
  boxes_mobilenet=priority_iou(boxes_mobilenet.tolist())
  #print(len(boxes_mobilenet))
  boxes_mobilenet=torch.from_numpy(np.array(boxes_mobilenet))
  result_mobilenet=iou(labels,boxes_mobilenet)
  area_result_mobilenet.append(result_mobilenet[0])
  accuracy_result_mobilenet.append(result_mobilenet[1])
  #break
  with torch.no_grad():
	  prediction =model_mobilenet_320([img])
	  boxes_mobilenet_320=prediction[0]["boxes"]
	  scores_mobilenet_320=prediction[0]["scores"]
  print("FOR MOBILENET 320")
  boxes_mobilenet_320=priority_iou(boxes_mobilenet_320.tolist())
  boxes_mobilenet_320=torch.from_numpy(np.array(boxes_mobilenet_320))
  result_mobilenet_320=iou(labels,boxes_mobilenet_320)
  area_result_mobilenet_320.append(result_mobilenet_320[0])
  accuracy_result_mobilenet_320.append(result_mobilenet_320[1])
  """for element in range(len(boxes)):
    if(scores[element]>0):
	      start_point=(boxes[element][0],boxes[element][1])
	      end_point=(boxes[element][2],boxes[element][3])
	      cv2.rectangle(image,start_point,end_point,(255,0,0),3)
  cv2.imwrite(output_path+image_array[detect], image)"""
  print(detect,"             ",image_array[detect])
  for element in boxes_resnet:
	  start_point=(element[0],element[1])
	  end_point=(element[2],element[3])
	  cv2.rectangle(image,start_point,end_point,(255,0,0),3)
  """for element in boxes_mobilenet:
	  start_point=(element[0],element[1])
	  end_point=(element[2],element[3])
	  cv2.rectangle(image,start_point,end_point,(255,255,0),3)
  for element in boxes_mobilenet_320:
	  start_point=(element[0],element[1])
	  end_point=(element[2],element[3])
	  cv2.rectangle(image,start_point,end_point,(0,255,0),3)"""
  #print(type(boxes_resnet))
  boxes=boxes_resnet.tolist()+boxes_mobilenet.tolist()+boxes_mobilenet_320.tolist()
  print("OVERALL")
  boxes=priority_iou(boxes)
  boxes=torch.from_numpy(np.array(boxes))
  result=iou(labels,boxes)
  area_result.append(result_mobilenet[0])
  accuracy_result.append(result[1])
  #print("final_result",result)
  #cv2_imshow( image)
  cv2.imwrite(output_path+image_array[detect], image)
  print(detect,"             ",image_array[detect])
  
  #break

hello
hello
hello
['B10D.jpg', 'B11A.jpg', 'B20F.jpg', 'B14A.jpg', 'B19.jpg', 'Bb32e.jpg', 'M51B.jpg', 'M53I.jpg', 'M52E.jpg', 'M52I.jpg', 'M52C.jpg', 'M53B.jpg']
FOR IMAGE : B10D.jpg
FOR RESNET
TOTAL OVERLAPPING: tensor(113.5053, dtype=torch.float64)
TOTAL DETECTIONS: 165
TOTAL LABELS:  156
TRUE POSITIVE:  143
FALSE NEGATIVE: 13
FALSE POSITIVE 22
FOR MOBILENET
[[993.087890625, 318.31219482421875, 1051.38525390625, 377.07586669921875], [975.5198974609375, 217.6319122314453, 1039.55224609375, 271.6505126953125], [307.9200134277344, 8.524090766906738, 370.4342956542969, 70.96492004394531], [683.0076904296875, 242.09231567382812, 736.8488159179688, 301.6480407714844], [1105.3187255859375, 789.6249389648438, 1159.1455078125, 845.4705200195312], [99.57831573486328, 491.8653259277344, 150.8326416015625, 542.4142456054688], [1181.736328125, 779.2373046875, 1234.14453125, 835.35107421875], [1169.4122314453125, 356.31768798828125, 1231.860595703125, 415.1800231933594], [865.6907958984375, 210.6

In [ ]:
print(result_resnet_50)
print(type(result_resnet_50))
print(area_result_resnet_50)

(tensor(0.7716, dtype=torch.float64), 0.7398373983739838)
<class 'tuple'>
tensor(0.7716, dtype=torch.float64)


In [ ]:
print(area_result_resnet_50)
print(area_result_mobilenet)
print(area_result_mobilenet_320)
print(area_result)
print(accuracy_result_resnet_50)
print(accuracy_result_mobilenet)
print(accuracy_result_mobilenet_320)
print(accuracy_result)
ar_r=0
ar_rr=0
ar_rm=0
ar_rm3=0
ac_r=0
ac_rr=0
ac_rm=0
ac_rm3=0
for i in area_result_resnet_50:
  ar_rr+=i/len(area_result_resnet_50)
for i in area_result_mobilenet:
  ar_rm+=i/len(area_result_mobilenet)
for i in area_result_mobilenet_320:
  ar_rm3+=i/len(area_result_mobilenet_320)
for i in area_result:
  ar_r+=i/len(area_result)
for i in accuracy_result_resnet_50:
  ac_rr+=i/len(accuracy_result_resnet_50)
for i in accuracy_result_mobilenet:
  ac_rm+=i/len(accuracy_result_mobilenet)
for i in accuracy_result_mobilenet_320:
  ac_rm3+=i/len(accuracy_result_mobilenet_320)
for i in accuracy_result:
  ac_r+=i/len(accuracy_result)
print(ar_rr)
print(ar_rm)
print(ar_rm3)
print(ar_r)
print(ac_rr)
print(ac_rm)
print(ac_rm3)
print(ac_r)

[tensor(0.7937, dtype=torch.float64), tensor(0.8419, dtype=torch.float64), tensor(0.7980, dtype=torch.float64), tensor(0.7887, dtype=torch.float64), tensor(0.7670, dtype=torch.float64), tensor(0.8092, dtype=torch.float64), tensor(0.8092, dtype=torch.float64), tensor(0.8113, dtype=torch.float64), tensor(0.8494, dtype=torch.float64), tensor(0.9124, dtype=torch.float64), tensor(0.8909, dtype=torch.float64), tensor(0.7952, dtype=torch.float64)]
[tensor(0.7476, dtype=torch.float64), tensor(0.7616, dtype=torch.float64), tensor(0.6893, dtype=torch.float64), tensor(0.7464, dtype=torch.float64), tensor(0.8559, dtype=torch.float64), tensor(0.8253, dtype=torch.float64), tensor(0.7230, dtype=torch.float64), tensor(0.7595, dtype=torch.float64), tensor(0.8942, dtype=torch.float64), tensor(0.7999, dtype=torch.float64), tensor(0.8663, dtype=torch.float64), tensor(0.7470, dtype=torch.float64)]
[tensor(0.8354, dtype=torch.float64), tensor(0.8494, dtype=torch.float64), tensor(0.6254, dtype=torch.float64)